# Server interaction

In [1]:
import paramiko
from pathlib import Path

## Using SSH to execute remote commands

**Important note:** some servers have a limit on the number of successful or unsuccessful logins for a time period.  If that limit is exceeded, your IP address will be banned for some time.  Hence be careful never to make a large number of connections over a short period of time!

Create a client, load the authentication keys, and connect to the system.

In [2]:
client = paramiko.SSHClient()
client.load_system_host_keys()

In [3]:
hostname = 'login1.hpc.kuleuven.be'

In [4]:
client.connect(hostname, username='vsc30140')

Execute an `ls` command on the remote host.

In [5]:
_, stdout, _ = client.exec_command('ls -l *.pbs')

Now the output of the remote `ls` command can be read from `stdout`.  Note that in this case we are ignoring both standard input and standard error.

In [6]:
for line in stdout:
    print(line.strip())

-rw-r----- 1 vsc30140 vsc30140 222 Aug 30 14:00 hello.pbs
-rw-r----- 1 vsc30140 vsc30140 112 Jun  1 09:21 hello_world.pbs
-rw-r----- 1 vsc30140 vsc30140 162 Nov  8 16:15 job.pbs
-rw-r----- 1 vsc30140 vsc30140 197 Oct 24 09:46 oe_test.pbs
-rw-r----- 1 vsc30140 vsc30140 174 Mar 15  2017 plot.pbs
-rw-r----- 1 vsc30140 vsc30140 125 Nov 13 13:35 test.pbs


### Dealing with errors

You can check the exit status of the command that was executed, the next one will not exit succesfully.  You can also access standard error.

In [7]:
_, stdout, stderr = client.exec_command('ls -l this_file_certainly_does_not_exists.txt')

In [8]:
stdout.channel.recv_exit_status()

2

In [9]:
print(''.join(stderr).rstrip())

ls: cannot access this_file_certainly_does_not_exists.txt: No such file or directory


### Using standard input

Do an `wc` on a local file that will be standard input to the remote command, reading the result back in.

In [10]:
stdin, stdout, _ = client.exec_command('wc')

Send the input to the remote `wc` by writing to `stdin`.  Once done, the channel for `stdin` should be shut down.

In [11]:
with open('julia_omp.f90', 'r') as file:
    for line in file:
        print(line, file=stdin, end='')
stdin.channel.shutdown_write()

Now the output from the command can be read from `stdout`.

In [12]:
print(''.join(stdout).rstrip())

    174     645    6270


Compare to the local result.

In [13]:
!wc julia_omp.f90

 174  645 6270 julia_omp.f90


In [14]:
client.close()

## Using SFTP to transfer files

In [15]:
client.connect(hostname, username='vsc30140')

Open an SFTP client to the remote host, and put a file on the system.

In [16]:
sftp_client = client.open_sftp()

Define paths to the input file and output file.

In [17]:
input_path = Path('julia_omp.f90')
output_path = Path('julia_omp_wc.txt')

In [18]:
_ = sftp_client.put(str(input_path), str(input_path))

Check whether the input file is on the remote host.

In [19]:
_, stdout, stderr = client.exec_command(f'ls -l {input_path}')
print(''.join(stdout).rstrip())
print(''.join(stderr).rstrip())

-rw-r--r-- 1 vsc30140 vsc30140 6270 Nov 14 11:36 julia_omp.f90



Compute the word count, redirecting the output to a file.

In [20]:
_, _, stderr = client.exec_command(f'wc {input_path} > {output_path}')

Show standard error.

In [21]:
print(''.join(stderr).rstrip())

Transfer the result back to the local host.

In [22]:
sftp_client.get(str(output_path), str(output_path))

In [23]:
!cat julia_omp_wc.txt

 174  645 6270 julia_omp.f90


Remove the local output file.

In [24]:
output_path.unlink()

Remove the remote input and output files.

In [25]:
_, _, _ = client.exec_command(f'rm {input_path} {output_path}')

Close the SFTP client and the SSH client.

In [26]:
sftp_client.close()
client.close()